In [61]:
import json
import re
import os
import calendar

import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go


pd.set_option('display.max_rows', 500)
months = calendar.month_abbr[1:]

In [2]:
file_path = r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly\embc2022.xlsx'
df = pd.read_excel(file_path,  sheet_name='Jan', header=None)

In [3]:
df[0] = df[0].fillna('')
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,"Date: January 9, 2022 Start: 08:30 Finish...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tide State: high Tide Movement: falling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Observers: Dave Lassmann, Judith Vetsch, Patri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Species: 48,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EC,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW
5,Pied-billed Grebe,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Double-crested Cormorant,17,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN
7,Pelagic Cormorant,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Great Blue Heron,15,NaN,NaN,NaN,NaN,NaN,1,3,NaN,...,NaN,9,NaN,NaN,1,NaN,NaN,NaN,1,NaN
9,Trumpeter Swan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
info_1 = df.iloc[0,0]
info_2 = df.iloc[1,0]
info_3 = df.iloc[2,0]
info_4 = df.iloc[3,0]
info = info_1 + '  ' + info_2 + '  ' + info_3 + '  ' + info_4 + '  '


In [5]:
observers_match = re.search(r'(Observers: +)([\w -,]+\w+)(   *)', info)
observers = observers_match.group(2)
observers

'Dave Lassmann, Judith Vetsch, Patrick MacNamara, Chris Murrell, Grant Danielson, Sophie Vielfaure, Karl Ricker, Tiffany Brunke, Kyle Kulas, Chris Dale'

In [6]:
equipment_match = re.search(r'(Equipment: +)([\w -]+\w+)(   *)', info)
equipment = equipment_match.group(2)
equipment

'scope - binoculars only'

In [7]:
tide_movement_match = re.search(r'(Tide Movement: +)([\w -]+\w+)(   *)', info)
tide_movement = tide_movement_match.group(2)
tide_movement

'falling'

In [8]:
tide_state_match = re.search(r'(Tide State: +)([\w -]+\w+)(   *)', info)
if tide_state_match:
    tide_state = tide_state_match.group(2)
    tide_state

In [9]:
sky_match = re.search(r'(Sky: +)([\w -]+\w+)(   *)', info)
sky = sky_match.group(2)
sky

'clear'

In [10]:
sea_state_match = re.search(r'(Sea State: +)([\w -]+\w+)(   *)', info)
sea_state = sea_state_match.group(2)
sea_state

'calm - ripple - chop'

In [11]:
info

'Date: January 9, 2022   Start: 08:30    Finish: 2:00     Sky: clear          Precip: none      Sea State:  calm - ripple - chop  Tide State: high      Tide Movement: falling      Equipment:  scope - binoculars only  Observers: Dave Lassmann, Judith Vetsch, Patrick MacNamara, Chris Murrell, Grant Danielson, Sophie Vielfaure, Karl Ricker, Tiffany Brunke, Kyle Kulas, Chris Dale    '

In [12]:
date_match = re.search(r'(Date:\s+)([\w -.,]+\w+)(   *)', info)
date = date_match.group(2)
date

'January 9, 2022'

In [13]:
start_match = re.search(r'(Start:\s+)(\w+:\w+)(   *)', info_1)
start = start_match.group(2)
start

'08:30'

In [14]:
finish_match = re.search(r'(Finish:\s+)(\w+:\w+)(   *)', info_1)
finish = finish_match.group(2)
finish

'2:00'

In [15]:
precip_match = re.search(r'(Precip: +)([\w -]+\w+)(   *)', info)
precip = precip_match.group(2)
precip

'none'

In [16]:
# get the header row
columns = df.columns
df[columns[2]] = df[columns[2]].astype('str')
header_idx = df[df[2].str.contains(r'OE', regex=True, na=False)].index
header_idx[0]

4

In [17]:
# get the notes index
columns = df.columns
df[columns[0]] = df[columns[0]].astype('str')
notes_idx = df[df[columns[0]].str.contains(r'Notes:', regex=True, na=False)].index
try:
    notes_idx[0]
except IndexError:
    pass


In [20]:
df1 = pd.read_excel(file_path,  sheet_name='Jan', header=header_idx[0])
columns = df1.columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Species: 48  52 non-null     object
 1   Total        55 non-null     object
 2   OE           10 non-null     object
 3   WD           8 non-null      object
 4   SR           11 non-null     object
 5   TD1          9 non-null      object
 6   TD2          12 non-null     object
 7   TD3          7 non-null      object
 8   EF1          15 non-null     object
 9   EF2          9 non-null      object
 10  CC           12 non-null     object
 11  CS           16 non-null     object
 12  EC           9 non-null      object
 13  EM           14 non-null     object
 14  BP           21 non-null     object
 15  NF1          13 non-null     object
 16  NF2          14 non-null     object
 17  SA           19 non-null     object
 18  IM           4 non-null      object
 19  MC1          22 non-null     ob

In [21]:
df1

,Species: 48,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EC,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW
0,Pied-billed Grebe,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Double-crested Cormorant,17,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN
2,Pelagic Cormorant,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Great Blue Heron,15,NaN,NaN,NaN,NaN,NaN,1,3,NaN,...,NaN,9,NaN,NaN,1,NaN,NaN,NaN,1,NaN
4,Trumpeter Swan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Canada Goose,356,NaN,50,NaN,NaN,NaN,NaN,67,NaN,...,28,15,14,NaN,6,NaN,NaN,7,NaN,NaN
6,American Wigeon,30,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,...,NaN,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mallard,677,NaN,120,NaN,NaN,NaN,NaN,105,NaN,...,66,70,37,9,3,NaN,NaN,1,26,6
8,Northern Pintail,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Green-winged Teal,17,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# drop notes rows
try:
    df1 = df1[:(notes_idx[0]-header_idx[0]-1)]
except IndexError:
    pass

In [23]:
# Drop nulls in the species column
df2 = df1.dropna(subset=[columns[0]])

In [24]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 54
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Species: 48  52 non-null     object
 1   Total        52 non-null     object
 2   OE           7 non-null      object
 3   WD           5 non-null      object
 4   SR           8 non-null      object
 5   TD1          6 non-null      object
 6   TD2          9 non-null      object
 7   TD3          4 non-null      object
 8   EF1          12 non-null     object
 9   EF2          6 non-null      object
 10  CC           9 non-null      object
 11  CS           13 non-null     object
 12  EC           6 non-null      object
 13  EM           11 non-null     object
 14  BP           18 non-null     object
 15  NF1          10 non-null     object
 16  NF2          11 non-null     object
 17  SA           16 non-null     object
 18  IM           1 non-null      object
 19  MC1          19 non-null     object


In [25]:
# Drop the totals column. This will be calculated later with pandas
df2 = df2.drop(columns='Total')

In [26]:
# fill species count nulls with zero. Then sort by species alphabetically 
df2 = df2.fillna(0)
df2 = df2.sort_values([columns[0]])

In [27]:
df2

,Species: 48,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,CC,...,EC,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW
17,Accipiter sp.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
20,American Coot,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
41,American Dipper,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
6,American Wigeon,0,0,0,0,0,0,12,0,8,...,0,8,2,0,0,0,0,0,0,0
27,Anna's Hummingbird,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
15,Bald Eagle,0,1,1,0,0,2,0,0,0,...,0,2,0,1,0,0,0,1,0,0
12,Barrrow's Goldeneye,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
28,Belted Kingfisher,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
37,Black-capped Chickadee,0,0,0,2,1,0,0,0,0,...,0,2,18,4,2,6,0,8,0,8
44,Bohemian Waxwing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
# Set the index to species and remove the index name
df2 = df2.set_index(columns[0])
df2.index.name=None

In [29]:
df2

,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,CC,CS,EC,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW
Accipiter sp.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
American Coot,0,0,0,0,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0
American Dipper,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
American Wigeon,0,0,0,0,0,0,12,0,8,0,0,8,2,0,0,0,0,0,0,0
Anna's Hummingbird,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Bald Eagle,0,1,1,0,0,2,0,0,0,0,0,2,0,1,0,0,0,1,0,0
Barrrow's Goldeneye,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
Belted Kingfisher,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Black-capped Chickadee,0,0,0,2,1,0,0,0,0,4,0,2,18,4,2,6,0,8,0,8
Bohemian Waxwing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [30]:
# Remove the totals rows
df2 = df2.drop('Totals')

In [31]:
# transpose to long form
df4 = pd.DataFrame()
for species in df2.index:
    species_count = df2.loc[species]
    species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date': '2022-06', 'species': species})
    df4 = pd.concat([df4, species_df])
df4

,id,count,date,species
0,OE,0,2022-06,Accipiter sp.
1,WD,0,2022-06,Accipiter sp.
2,SR,0,2022-06,Accipiter sp.
3,TD1,0,2022-06,Accipiter sp.
4,TD2,0,2022-06,Accipiter sp.
...,...,...,...,...
15,SA,1,2022-06,Winter Wren (Pacific)
16,IM,0,2022-06,Winter Wren (Pacific)
17,MC1,0,2022-06,Winter Wren (Pacific)
18,MC2,0,2022-06,Winter Wren (Pacific)


In [32]:
# Total number of species for individual areas
total_count = df2.gt(0).sum(axis=0)
df3 = pd.DataFrame({'id': total_count.index, 'counts': total_count.values, 'date':'2022-06', 'species': 'Total Species'})
df3

,id,counts,date,species
0,OE,6,2022-06,Total Species
1,WD,4,2022-06,Total Species
2,SR,7,2022-06,Total Species
3,TD1,5,2022-06,Total Species
4,TD2,8,2022-06,Total Species
5,TD3,3,2022-06,Total Species
6,EF1,11,2022-06,Total Species
7,EF2,5,2022-06,Total Species
8,CC,8,2022-06,Total Species
9,CS,12,2022-06,Total Species


# Load Data from excel sheets

In [62]:
def read_excel_file(year):

    excel_df = pd.DataFrame()
    file_path = os.path.join(r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly', f'embc{year}.xlsx')
    
    for month in months:
        print(f'{year}-{month}')
        df = pd.read_excel(file_path,  sheet_name=month, header=None)
        df[0] = df[0].astype('str')
        # find header row and reload file from that row
        header_idx = df[df[2].str.contains(r'OE', regex=True, na=False)].index
        # find notes_idx
        notes_idx = df[df[0].str.contains(r'Notes:', regex=True, na=False)].index
        
        df1 = pd.read_excel(file_path,  sheet_name=month, header=header_idx[0])
        columns = df1.columns

        # drop notes rows if there are any
        try:
            df1 = df1[:(notes_idx[0]-header_idx[0]-1)]
        except IndexError:
            pass
        
        # find extra header rows and remove
        df1[columns[2]] = df1[columns[2]].astype('str')
        extra_header_idx = df1[df1[columns[2]].str.contains(r'OE', regex=True, na=False)].index
        if len(extra_header_idx) > 0:
            for header_row_idx in extra_header_idx:
                df1 = df1.drop(header_row_idx)
        
        df2 = df1.dropna(subset=[columns[0]])
        df2 = df2.drop(columns='Total')
        df2[df2.columns] = df2[df2.columns].fillna(0)
        df2 = df2.sort_values([columns[0]])
        total_idx = df2[df2[columns[0]].str.contains(r'Totals?|Overall.*', regex=True, na=False)].index
        df2 = df2.drop(total_idx)

        # drop species that start with asterisk
        asterisk_idx = df2[df2[columns[0]].str.contains(r'^\*', regex=True, na=False)].index
        df2 = df2.drop(asterisk_idx)
        df2 = df2.set_index(columns[0])
        df2.index.name=None

        # individual species count
        for species in df2.index:
            species_count = df2.loc[species]
            species_count = pd.to_numeric(species_count, errors='coerce').fillna(0)
            species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date':f'{month}-{year}', 'year': year, 'month': month,'species': species})
            excel_df = pd.concat([excel_df, species_df])

    excel_df['count'] = excel_df['count'].astype('int64')
    return excel_df

In [63]:
excel_df = pd.DataFrame()
for year in range(1999,2023):
    excel_year_df = read_excel_file(year)
    excel_df = pd.concat([excel_df, excel_year_df])


1999-Jan


1999-Feb
1999-Mar
1999-Apr
1999-May
1999-Jun
1999-Jul
1999-Aug
1999-Sep
1999-Oct
1999-Nov
1999-Dec
2000-Jan
2000-Feb
2000-Mar
2000-Apr
2000-May
2000-Jun
2000-Jul
2000-Aug
2000-Sep
2000-Oct
2000-Nov
2000-Dec
2001-Jan
2001-Feb
2001-Mar
2001-Apr
2001-May
2001-Jun
2001-Jul
2001-Aug
2001-Sep
2001-Oct
2001-Nov
2001-Dec
2002-Jan
2002-Feb
2002-Mar
2002-Apr
2002-May
2002-Jun
2002-Jul
2002-Aug
2002-Sep
2002-Oct
2002-Nov
2002-Dec
2003-Jan
2003-Feb
2003-Mar
2003-Apr
2003-May
2003-Jun
2003-Jul
2003-Aug
2003-Sep
2003-Oct
2003-Nov
2003-Dec
2004-Jan
2004-Feb
2004-Mar
2004-Apr
2004-May
2004-Jun
2004-Jul
2004-Aug
2004-Sep
2004-Oct
2004-Nov
2004-Dec
2005-Jan
2005-Feb
2005-Mar
2005-Apr
2005-May
2005-Jun
2005-Jul
2005-Aug
2005-Sep
2005-Oct
2005-Nov
2005-Dec
2006-Jan
2006-Feb
2006-Mar
2006-Apr
2006-May
2006-Jun
2006-Jul
2006-Aug
2006-Sep
2006-Oct
2006-Nov
2006-Dec
2007-Jan
2007-Feb
2007-Mar
2007-Apr
2007-May
2007-Jun
2007-Jul
2007-Aug
2007-Sep
2007-Oct
2007-Nov
2007-Dec
2008-Jan
2008-Feb
2008-Mar
2008-Apr
2

In [35]:
sorted(list(excel_df['species'].unique()))

[' Dark-eyed Junco',
 'Accipiter sp.',
 'Accipiter, sp.',
 'Accipter sp.',
 'Alcid sp.',
 'American Bittern',
 'American Coot',
 'American Crow',
 'American Crow x Northwestern Crow',
 'American Dipper',
 'American Goldfinch',
 'American Kestrel',
 'American Pipit',
 'American Redstart',
 'American Robbin',
 'American Robin',
 'American Tree Sparrow',
 'American Widgeon',
 'American Wigeion',
 'American Wigeon',
 'American redstart',
 "Anna's Hummingbird",
 'Anna’s Humming Bird',
 'Anna’s Hummingbird',
 'Back Swift',
 "Baird's Sandpiper",
 'Bald Eagle',
 'Band-tailed Pigeon',
 'Bank Swallow',
 'Barn Swallow',
 'Barred Owl',
 "Barrow's Goldeneye",
 'Barrows Goldeneye',
 "Barrrow's Goldeneye",
 'Belted Kingfisher',
 'Black Oystercatcher',
 'Black Swift',
 'Black Turnstone',
 'Black-and-White Warbler',
 'Black-capped Chickadee',
 'Black-headed Grosbeak',
 'Black-throated Gray Warbler',
 'Black-throated Grey Warbler',
 'Blackbird sp.',
 'Blacked-headed Grosbeak',
 'Blue Grouse',
 'Blue Gro

In [64]:
# Replacements Dictionary to correct spelling from excel sheets
replacements = {
    'Accipter sp.': 'Accipiter sp.',
    'Accipiter, sp.': 'Accipiter sp.',
    'American Crow x Northwestern Crow': 'American Crow',
    'American Robbin': 'American Robin',
    'American Wigeion': 'American Wigeon',
    'American Widgeon': 'American Wigeon',
    'Anna’s Humming Bird': "Anna's Hummingbird",
    'Anna’s Hummingbird': "Anna's Hummingbird",
    'Back Swift': 'Black Swift',
    'Barrows Goldeneye': "Barrow's Goldeneye",
    "Barrrow's Goldeneye": "Barrow's Goldeneye",
    'Blacked-headed Grosbeak': 'Black-headed Grosbeak',
    'Black-throated Grey Warbler': 'Black-throated Gray Warbler',
    'Blue Grouse (Sooty)': 'Sooty Grouse',
    'Brandt': "Brandt's Cormorant",
    "Brant's Cormorant": "Brandt's Cormorant",
    'Bustit': 'Bushtit',
    'California (Western) Scrub-Jay': 'California Scrub-Jay',
    'Cedar Wagwing': 'Cedar Waxwing',
    'Chestnut-back Chickadee': 'Chestnut-backed Chickadee',
    'Common Yellowthoat': 'Common Yellowthroat',
     'Cormorant, sp.': 'Cormorant sp.',
    ' Dark-eyed Junco': 'Dark-eyed Junco',
    'Double Crested Cormorant': 'Double-crested Cormorant',
    'Doubel-crested Cormorant': 'Double-crested Cormorant',
    'Doule-crested Cormorant': 'Double-crested Cormorant',
    'Duck, sp': 'Duck sp.',
    'Eurasian Collar-Dove': 'Eurasian Collared Dove',
    'Eurasian Collared-Dove': 'Eurasian Collared Dove',
    'European Collared-Dove':'Eurasian Collared Dove',
    'Eurasian Widgeon': 'Eurasian Wigeon',
    'Flycatcher, sp.': 'Flycatcher sp.',
    'Glaucous-winged x Herring Gull': 'Gull, Glaucous-winged x Herring',
    'Gul sp.': 'Gull sp.',
    'Gull, sp.': 'Gull sp.',
    'Gull Hybrid': 'Gull, Hybrid',
    'Hybrid gull': 'Gull, Hybrid',
    'Glaucous-w x Western Gull': 'Gull, Glaucous-winged x Western',
    'Glaucous-wing x Western': 'Gull, Glaucous-winged x Western',
    'Glaucous-wing x Western G.': 'Gull, Glaucous-winged x Western',
    'Glaucous-wing.x Western G.': 'Gull, Glaucous-winged x Western',
    'Glaucous-winged Gull': 'Gull, Glaucous-winged x Western',
    'Glaucous-winged Gull x Western': 'Gull, Glaucous-winged x Western',
    'Glaucous-winged x Western': 'Gull, Glaucous-winged x Western',
    'Glaucous-wingxWestern Gull': 'Gull, Glaucous-winged x Western',
    'Gull, Western x Glaucous-wing': 'Gull, Glaucous-winged x Western',
    'Gull, Western x Glaucous-winged': 'Gull, Glaucous-winged x Western',
    'Gull,Glaucous-winged x Western': 'Gull, Glaucous-winged x Western',
    'Gull, G-W x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, G-w. x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, G.w. x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, Gl.-winged x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, Glaucous x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, Glaucous-w. x Western': 'Gull, Glaucous-wingded x Western',
    'Gull, Western x Glaucous W.': 'Gull, Glaucous-wingded x Western',
    'Gull, Western x Glaucous-w.': 'Gull, Glaucous-wingded x Western',
    'Hybrid Gull,Glaucous-w. x W.': 'Gull, Glaucous-wingded x Western',
    'Goldeneye, sp.': 'Goldeneye sp.',
    'Harbour Seals': 'Harbour Seal',
    'Seal': 'Harbour Seal',
    'Seals': 'Harbour Seal',
    'Hawk Species': 'Hawk sp.',
    'Hawk, sp.': 'Hawk sp.',
    'House Sparow': 'House Sparrow',
    "Lewis' Woodpecker": "Lewis's Woodpecker",
    "Lincoln's Sparow": "Lincoln's Sparrow",
    'Lincoln’s Sparrow': "Lincoln's Sparrow",
    "MacGillivray's  Warbler": "MacGillivray's Warbler",
    "McGillivray's Warbler": "MacGillivray's Warbler",
    'Marsh Wren ': 'Marsh Wren',
    'Norther Harrier': 'Northern Harrier',
    'Njorthern Harrier': 'Northern Harrier',
    'Northern Pygmy-owl': 'Northern Pygmy Owl',
    'Northern Pygmy-Owl': 'Northern Pygmy Owl',
    'Northwestern Crow': 'American Crow',
    'Nthn Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winges Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-Winged Swallow': 'Northern Rough-winged Swallow',
    'Nthrn. Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Olive-sided Flcatcher': 'Olive-sided Flycatcher',
    'Pacific-Slope Flycatcher': 'Pacific-slope Flycatcher',
    'Pacific Slope Flycatcher': 'Pacific-slope Flycatcher',
    'Passerine sp': 'Passerine sp.',
    'Pelgagic Cormorant': 'Pelagic Cormorant',
    'Peregrin Falcon': 'Peregrine Falcon',
    'Peregrine': 'Peregrine Falcon',
    'Peregrine Falacon': 'Peregrine Falcon',
    'Pied -billed Grebe': 'Pied-billed Grebe',
    'Pied Bil Grebe': 'Pied-billed Grebe',
    'Pileated Wodpecker': 'Pileated Woodpecker',
    'Pileated WoodPecker': 'Pileated Woodpecker',
    'Pileated Woodpeckers': 'Pileated Woodpecker',
    'Red Throated Loon': 'Red-throated Loon',
    'Red-Breasted Nuthatch': 'Red-breasted Merganser',
    'Red--breasted Merganser': 'Red-breasted Merganser',
    'Red-Breasted Nuthatch': 'Red-breasted Nuthatch',
    'Red-breaste Nuthatch': 'Red-breasted Nuthatch',
    'Red-tailed hawk': 'Red-tailed Hawk',
    'Ring-biled Gull': 'Ring-billed Gull',
    'Rock Dove': 'Pigeon',
    'Rock Dove (Pigeon)': 'Pigeon',
    'Rock Dove (Rock Pigeon)': 'Pigeon',
    'Rock Pigeon': 'Pigeon',
    'Ruby-crowned Kinglety': 'Ruby-crowned Kinglet',
    'Rufous Hummngbird': 'Rufous Hummingbird',
    'Sandpiper, sp.': 'Sandpiper sp.',
    'Savannah': 'Savannah Sparrow',
    'Scaup, sp.': 'Scaup sp.',
    'Scaup Sp.': 'Scaup sp.',
    'Scaup sp': 'Scaup sp.',
    'Semi-palmated Sandpiper':  'Semipalmated Sandpiper',
    'Sharp-Shinned Hawk': 'Sharp-shinned Hawk', 
    'Sharp-shinned Hawk*': 'Sharp-shinned Hawk',
    'Short-blled Gull': 'Short-billed Gull',
    'Shorebird, sp.': 'Shorebird sp.',
    'Sooty (Blue) Grouse': 'Sooty Grouse',
    'Spotted Sandiper': 'Spotted Sandpiper',
    "Stellar's Jay": "Steller's Jay",
    'Steller’s Jay': "Steller's Jay",
    'Stellar’s Jay': "Steller's Jay",
    'Swainson’s Thrush': "Swainson's Thrush",
    'Teal, sp.': 'Teal sp.',
    'Townsend’s Warbler': "Townsend's Warbler",
    "Townsend's warbler": "Townsend's Warbler",
    "Vaux' Swift": "Vaux's Swift",
    'Vaux’s Swift': "Vaux's Swift",
    'Warling Vireo': 'Warbling Vireo',
    'Western Wood-Pewee': 'Western Wood Pewee',
    'Western Wood-peewee': 'Western Wood Pewee',
    'Western Wood-pewee': 'Western Wood Pewee',
    'White-Winged Crossbill': 'White-winged Crossbill',
    'Winter (Pacific) Wren': 'Pacific Wren',
    'Winter Wren': 'Pacific Wren',
    'Winter Wren (Pacific Wren)': 'Pacific Wren',
    'Winter Wren (Pacific)': 'Pacific Wren',
    "Wison's Warbler": "Wilson's Warbler",
    'Woopecker sp.': 'Woodpecker sp.',
    'Woodpecker sp': 'Woodpecker sp.',
    'chickadee sp.': 'Chickadee sp.',
    'finch sp.': 'Finch sp.',
    'gull sp.': 'Gull sp.',
    'hummingbird sp.': 'Hummingbird sp.',
    'sparrow sp.': 'Sparrow sp.',
    'swallow sp.': 'Swallow sp.',
    'blackbird sp.': 'Blackbird sp.',
    'chickadee sp': 'Chickadee sp.',
    'chickadee, sp.': 'Chickadee sp.',
    'cormorant sp.': 'Cormorant sp.',
    'cormorant, sp.': 'Cormorant sp.',
    'crow sp.': 'Crow sp.',
    'diving duck sp.': 'Diving duck sp.',
    'duck sp.': 'Duck sp.', 
    'finch, sp.': 'Finch sp.',
    'flycatcher sp': 'Flycatcher sp.',
    'flycatcher sp.': 'Flycatcher sp.',
    'goldeneye, sp.': 'Goldeneye sp.',
    'grebe sp.': 'Grebe sp.',
    'grouse sp.': 'Grouse sp.',
    'gull, sp.': 'Gull sp.',
    'harbour seals': 'Harbour Seal',
    'hawk sp.': 'Hawk sp.',
    'hawk, sp.': 'Hawk sp.',
    'kinglet sp.': 'Kinglet sp.',
    'large hawk sp.': 'Large Hawk sp.',
    'loon sp.': 'Loon sp.',
    'loon, sp.': 'Loons sp.',
    'merganser sp.': 'Merganser sp.',
    'passerine': "Passerine sp.",
    'passerine sp.': 'Passerine sp.',
    'sandpiper sp.': 'Sandpiper sp.',
    'sandpiper, sp.': 'Sandpiper sp.',
    'scaup, sp.': 'Scaup sp.',
    'scoter sp.': 'Scoter sp.',
    'shorebird sp.': 'Shorebird sp.',
    'swallow sp': 'Swallow sp.',
    'swift sp.': 'Swift sp.',
    'teal sp.': 'Teal sp.',
    'warbler sp.': 'Warbler sp.',
    'woodpecker sp.': 'Woodpecker sp.'
}

In [65]:
df = excel_df.replace(replacements)

In [66]:
spell_checked_df = df.replace(replacements)

In [67]:
sorted(list(spell_checked_df['species'].unique()))

['Accipiter sp.',
 'Alcid sp.',
 'American Bittern',
 'American Coot',
 'American Crow',
 'American Dipper',
 'American Goldfinch',
 'American Kestrel',
 'American Pipit',
 'American Redstart',
 'American Robin',
 'American Tree Sparrow',
 'American Wigeon',
 'American redstart',
 "Anna's Hummingbird",
 "Baird's Sandpiper",
 'Bald Eagle',
 'Band-tailed Pigeon',
 'Bank Swallow',
 'Barn Swallow',
 'Barred Owl',
 "Barrow's Goldeneye",
 'Belted Kingfisher',
 'Black Oystercatcher',
 'Black Swift',
 'Black Turnstone',
 'Black-and-White Warbler',
 'Black-capped Chickadee',
 'Black-headed Grosbeak',
 'Black-throated Gray Warbler',
 'Blackbird sp.',
 'Blue Grouse',
 'Blue-winged Teal',
 'Bohemian Waxwing',
 "Bonaparte's Gull",
 "Brandt's Cormorant",
 "Brewer's Blackbird",
 'Brown Creeper',
 'Brown-headed Cowbird',
 'Budgerigar (Budgie)',
 'Bufflehead',
 "Bullock's Oriole",
 'Bushtit',
 'Buteo sp.',
 'Cackling Goose',
 'California Gull',
 'California Scrub-Jay',
 'Canada Goose',
 'Canvasback',
 

In [68]:
spell_checked_df[spell_checked_df['species'] == 'Canada Goose:  1 j in BP ']

,id,count,date,year,month,species


In [69]:
# Calculate total individual bird count by area and concat
areas_total_bird_count_df = spell_checked_df.groupby(['id','year','month', 'date']).sum('count').reset_index()
areas_total_bird_count_df['species'] = 'Total Bird Count'
areas_total_added_df = pd.concat([spell_checked_df, areas_total_bird_count_df])

In [70]:
areas_total_bird_count_df

,id,year,month,date,count,species
0,BP,1999,Apr,Apr-1999,155,Total Bird Count
1,BP,1999,Aug,Aug-1999,309,Total Bird Count
2,BP,1999,Dec,Dec-1999,29,Total Bird Count
3,BP,1999,Feb,Feb-1999,59,Total Bird Count
4,BP,1999,Jan,Jan-1999,51,Total Bird Count
...,...,...,...,...,...,...
5761,WD,2022,Mar,Mar-2022,40,Total Bird Count
5762,WD,2022,May,May-2022,15,Total Bird Count
5763,WD,2022,Nov,Nov-2022,7,Total Bird Count
5764,WD,2022,Oct,Oct-2022,8,Total Bird Count


In [71]:
# Calculate total birds across all areas
all_area_total_bird = areas_total_added_df.groupby(['year','month', 'date', 'species']).sum('count').reset_index()
all_area_total_bird['id'] = 'ALL'
all_area_total_added_df = pd.concat([areas_total_added_df, all_area_total_bird])

In [72]:
all_area_total_bird

,year,month,date,species,count,id
0,1999,Apr,Apr-1999,American Coot,4,ALL
1,1999,Apr,Apr-1999,American Crow,106,ALL
2,1999,Apr,Apr-1999,American Pipit,8,ALL
3,1999,Apr,Apr-1999,American Robin,159,ALL
4,1999,Apr,Apr-1999,American Wigeon,22,ALL
...,...,...,...,...,...,...
17815,2022,Sep,Sep-2022,Violet-green Swallow,1,ALL
17816,2022,Sep,Sep-2022,Virginia Rail,1,ALL
17817,2022,Sep,Sep-2022,White-crowned Sparrow,55,ALL
17818,2022,Sep,Sep-2022,Willow Flycatcher,1,ALL


In [73]:
all_area_total_added_df[(all_area_total_added_df['species']=='Dark-eyed Junco') & (all_area_total_added_df['year']==2013)]

,id,count,date,year,month,species
0,OE,0,Jan-2013,2013,Jan,Dark-eyed Junco
1,WD,0,Jan-2013,2013,Jan,Dark-eyed Junco
2,SR,0,Jan-2013,2013,Jan,Dark-eyed Junco
3,TD1,0,Jan-2013,2013,Jan,Dark-eyed Junco
4,TD2,0,Jan-2013,2013,Jan,Dark-eyed Junco
5,TD3,0,Jan-2013,2013,Jan,Dark-eyed Junco
6,EF1,0,Jan-2013,2013,Jan,Dark-eyed Junco
7,EF2,0,Jan-2013,2013,Jan,Dark-eyed Junco
8,CC,10,Jan-2013,2013,Jan,Dark-eyed Junco
9,CS,8,Jan-2013,2013,Jan,Dark-eyed Junco


In [74]:
# Calculate the number of species in alls area
# First remove "Total Bird Count" from totals
all_area_species = all_area_total_added_df[all_area_total_added_df['species']!='Total Bird Count']
all_area_total_species = all_area_species.groupby(['id', 'date', 'year', 'month']).apply(lambda x: x['count'].gt(0).sum(axis=0)).reset_index(name='count')
all_area_total_species['species'] = 'Total Species Count'
areas_and_species_totals_added_df = pd.concat([all_area_total_added_df, all_area_total_species])


In [75]:
areas_and_species_totals_added_df[(areas_and_species_totals_added_df['species']=='Total Species Count') & (areas_and_species_totals_added_df['year']==2013)]

,id,count,date,year,month,species
14,ALL,74,Apr-2013,2013,Apr,Total Species Count
38,ALL,56,Aug-2013,2013,Aug,Total Species Count
62,ALL,41,Dec-2013,2013,Dec,Total Species Count
86,ALL,57,Feb-2013,2013,Feb,Total Species Count
110,ALL,63,Jan-2013,2013,Jan,Total Species Count
134,ALL,57,Jul-2013,2013,Jul,Total Species Count
158,ALL,71,Jun-2013,2013,Jun,Total Species Count
182,ALL,60,Mar-2013,2013,Mar,Total Species Count
206,ALL,74,May-2013,2013,May,Total Species Count
230,ALL,56,Nov-2013,2013,Nov,Total Species Count


In [76]:
# test_final3[test_final3['month'].isnull()]
len(areas_and_species_totals_added_df.species.unique())

295

In [77]:
# sort data with months in the proper order
df_loaded = areas_and_species_totals_added_df
df_loaded['month'] = pd.Categorical(df_loaded['month'], months)
df_loaded = df_loaded.sort_values(['year', 'month','species']).reset_index(drop=True)


In [78]:
df_loaded[(df_loaded['year']==2013) & (df_loaded['id']=='ALL') & (df_loaded['species']=='Dark-eyed Junco')]


,id,count,date,year,month,species
222273,ALL,168,Jan-2013,2013,Jan,Dark-eyed Junco
223575,ALL,88,Feb-2013,2013,Feb,Dark-eyed Junco
224793,ALL,115,Mar-2013,2013,Mar,Dark-eyed Junco
226137,ALL,95,Apr-2013,2013,Apr,Dark-eyed Junco
229454,ALL,2,Jun-2013,2013,Jun,Dark-eyed Junco
234683,ALL,33,Oct-2013,2013,Oct,Dark-eyed Junco
236005,ALL,53,Nov-2013,2013,Nov,Dark-eyed Junco
237118,ALL,74,Dec-2013,2013,Dec,Dark-eyed Junco


In [79]:
# Fill in data counts of 0 for birds that were not seen in a particular month
zero_counts_added = df_loaded.copy()
for year in range(1999,2023):
    year_all_total = df_loaded[(df_loaded['year']==year) & (df_loaded['id']=='ALL')]

    # Build template to use for merging 0 counts with existing data. The 
    # dataframe has no count column. When merged with existing data, rows with 
    # a n/a in the count column had no previous count and can be filled with 0
    merge_template = pd.DataFrame()
    for specie in year_all_total.species.unique():
        temp = pd.DataFrame({'id': 'ALL', 'date': [f'{month}-{year}' for month in months], 'year': year, 'month':months, 'species': specie})
        # if specie == 'Dark-eyed Junco':
        #     print(temp)
        merge_template = pd.concat([merge_template, temp])
        
    # merge with the count and fill na with 0
    merge_df = merge_template.merge(year_all_total, how='left', on=['id', 'date', 'year', 'month', 'species']).fillna(0)

    # extract just the newly created zero counts and add to existing data
    zero_counts = merge_df[merge_df['count']==0]
    zero_counts_added = pd.concat([zero_counts_added, zero_counts])


In [80]:
zero_counts_added[(zero_counts_added['species']=='Dark-eyed Junco') & (zero_counts_added['year']==2013) & (zero_counts_added['id']=='ALL')]

,id,count,date,year,month,species
222273,ALL,168.0,Jan-2013,2013,Jan,Dark-eyed Junco
223575,ALL,88.0,Feb-2013,2013,Feb,Dark-eyed Junco
224793,ALL,115.0,Mar-2013,2013,Mar,Dark-eyed Junco
226137,ALL,95.0,Apr-2013,2013,Apr,Dark-eyed Junco
229454,ALL,2.0,Jun-2013,2013,Jun,Dark-eyed Junco
234683,ALL,33.0,Oct-2013,2013,Oct,Dark-eyed Junco
236005,ALL,53.0,Nov-2013,2013,Nov,Dark-eyed Junco
237118,ALL,74.0,Dec-2013,2013,Dec,Dark-eyed Junco
256,ALL,0.0,May-2013,2013,May,Dark-eyed Junco
258,ALL,0.0,Jul-2013,2013,Jul,Dark-eyed Junco


In [81]:
# Sort data
df = zero_counts_added
df['month'] = pd.Categorical(df['month'], months)
df = df.sort_values(['year', 'month','species']).reset_index(drop=True)
df['count'] = df['count'].astype('int64')

In [82]:
len(df.species.unique())

295

In [83]:
# Stats for data
def get_stats_df(df):
    """Return a dataframe with some statistics"""
    stats_df = df[df['id']=='ALL']
    stats_df = stats_df.groupby(
        ['month', 'species']
        ).agg(
            mean=('count', np.mean),
            median=('count', np.median),
            std=('count', np.std),
            min=('count', np.min),
            max=('count', np.max)
        ).reset_index()

    stats_df['month'] = pd.Categorical(stats_df['month'], months)
    stats_df = stats_df.sort_values(['month','species']).reset_index(drop=True)
    
    return stats_df

In [84]:
get_stats_df(df[(df['species']== "Dark-eyed Junco") & (df['id']=='ALL')])

,month,species,mean,median,std,min,max
0,Jan,Dark-eyed Junco,99.333333,90.0,62.504029,12,217
1,Feb,Dark-eyed Junco,67.666667,67.5,38.631668,4,156
2,Mar,Dark-eyed Junco,71.041667,59.0,51.891592,0,227
3,Apr,Dark-eyed Junco,101.875000,92.5,98.383152,1,329
4,May,Dark-eyed Junco,15.750000,2.0,31.801354,0,108
5,Jun,Dark-eyed Junco,0.958333,0.0,2.053188,0,9
6,Jul,Dark-eyed Junco,0.250000,0.0,0.607919,0,2
7,Aug,Dark-eyed Junco,0.416667,0.0,0.928611,0,4
8,Sep,Dark-eyed Junco,1.333333,1.0,1.372610,0,4
9,Oct,Dark-eyed Junco,49.041667,42.0,37.835432,6,137


In [85]:
# 2004 has no data for Sep and Dec so those months are removed
df[((df['year']==2004) & (df['month']=='Sep') )| ((df['year']==2004) & (df['month']=='Dec'))]

,id,count,date,year,month,species
99068,ALL,0,Sep-2004,2004,Sep,Accipiter sp.
99069,ALL,0,Sep-2004,2004,Sep,American Coot
99070,OE,0,Sep-2004,2004,Sep,American Crow
99071,WD,0,Sep-2004,2004,Sep,American Crow
99072,SR,0,Sep-2004,2004,Sep,American Crow
...,...,...,...,...,...,...
104736,ALL,0,Dec-2004,2004,Dec,Wilson's Warbler
104737,ALL,0,Dec-2004,2004,Dec,Wood Duck
104738,ALL,0,Dec-2004,2004,Dec,Woodpecker sp.
104739,ALL,0,Dec-2004,2004,Dec,Yellow Warbler


In [86]:
drop_idx = df[((df['year']==2004) & (df['month']=='Sep') )| ((df['year']==2004) & (df['month']=='Dec'))].index
df = df.drop(drop_idx)

# Save data

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405636 entries, 0 to 408656
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   id       405636 non-null  object  
 1   count    405636 non-null  int64   
 2   date     405636 non-null  object  
 3   year     405636 non-null  int64   
 4   month    405636 non-null  category
 5   species  405636 non-null  object  
dtypes: category(1), int64(2), object(3)
memory usage: 19.0+ MB


In [88]:
df.to_csv('count_data_1999_2022.csv')

# App

read areas and mapbox token

In [3]:
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\data\areas.json"
) as areas_file:
    areas = json.load(areas_file)
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\.mapbox_token"
) as token_file:
    token = token_file.read()



Get Data from csv

In [4]:
df = pd.read_csv(r'C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\bird_count\data\count_data.csv', index_col=0)

In [7]:
app = JupyterDash(__name__)

app.layout = html.Div(
    id="app-container",
    children=[
        html.Div(
            id="sidebar-container",
            children=[
                html.H1("Squamish Monthly Bird Count"),
                dcc.Tabs(
                    id="tabs",
                    value="tab-graph",
                    children=[
                        dcc.Tab(label="Graph", value="tab-graph"),
                        dcc.Tab(label="Map", value="tab-map"),
                    ],
                ),
                html.P("Select Species:"),
                        dcc.Dropdown(
                            id="species-dropdown",
                            options=sorted(df["species"].unique()),
                            value="Total Species Count",
                            clearable=False,
                        ),
                html.Div(
                    id='sidebar-content',
                    ),
            ],
        ),
        html.Div(
            id="content-container",
        ),
    ],
)

@app.callback(
    Output('sidebar-content', 'children'),
    Input('tabs', 'value')
)
def render_sidebar_content(tab):
    if tab == 'tab-map':
        return []
    elif tab == 'tab-graph':
        return [
            # html.P("Line Shape:"),
            dcc.RadioItems(
                ['spline', 'linear',],
                'spline',
                id='line-shape-radio',
                inline=True
            ),
            dcc.Checklist(
                ['Average', 'Standard Deviation'],
                [],
                id='average-checklist'
            )
        ]


@app.callback(
    Output('content-container', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-map':
        return dcc.Graph(
            id="count-map",
            config=dict(responsive=True),
        )
    elif tab == 'tab-graph':
        return dcc.Graph(
            id="count-graph",
            config=dict(responsive=True),
        )

@app.callback(
    Output("count-map", "figure"), 
    Input("species-dropdown", "value")
)
def update_map(species):
    dff = df[df["species"] == species]
    
    fig = px.choropleth_mapbox(
        dff,
        geojson=areas,
        locations="id",
        featureidkey="properties.id",
        color="count",
        color_continuous_scale="Purples",
        range_color=(0, dff["count"].max()),
        zoom=12.5,
        center={"lat": 49.7, "lon": -123.15},
        opacity=0.5,
        labels={"count": "Count", "id": "Area"},
        animation_frame="date",
        template="plotly_dark",
    )
    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
        mapbox_accesstoken=token,
        mapbox_style="satellite-streets",
    )
    if fig["layout"]["updatemenus"]:
        fig["layout"]["updatemenus"][0]["pad"] = dict(r=20, t=25)
        fig["layout"]["sliders"][0]["pad"] = dict(r=0, t=0, b=20)

    return fig

@app.callback(
    Output("count-graph", "figure"), 
    Input("species-dropdown", "value"),
    Input("line-shape-radio", "value"),
    Input('average-checklist', 'value'),
)
def update_graph(species, line_shape, average_checklist):
    dff = df[(df['species']== species) & (df['id']=='ALL')]
    stats_df = get_stats_df(dff)

    fig = px.line(
        dff,
        x='month',
        y='count',
        category_orders= {'month': months},
        color="year",
        color_discrete_sequence=px.colors.qualitative.Light24,
        template="plotly_dark",
        markers=True,
        line_shape=line_shape,
    )

    if 'Average' in average_checklist:
        # Average
        fig.add_trace(
            go.Scatter(
                x=stats_df['month'],
                y=stats_df['mean'], 
                mode="lines",
                line_shape=line_shape,
                name='Average',
                line={'width':4, 'color':'white'},
                showlegend=False
            )
        )
    
    if 'Standard Deviation' in average_checklist:
        # Standard deviation line
        average_plus_std = list(stats_df['mean']+stats_df['std'])
        average_minus_std = list(stats_df['mean']-stats_df['std'])
        rev_average_minus_std = average_minus_std[::-1]
        rev_average_minus_std = [x if x > 0 else 0 for x in rev_average_minus_std]

        fig.add_trace(
            go.Scatter(
                x=months+months[::-1],
                y=average_plus_std+rev_average_minus_std,
                fill='toself',
                fillcolor='rgba(255,255,255,0.3)',
                line_color='rgba(255,255,255,0)',
                mode="lines",
                line_shape=line_shape,
                name='Standard Deviation',
                line={'width':4, 'color':'white'},
                showlegend=False
            )
        )

    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
    )

    fig.data = fig.data[::-1]
    return fig



if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
